# Pandas II

## Primera exploración

Lo primero que uno tiene que hacer cuando recibe un dataset es explorar qué contiene: ser curiosos! 

Algunas preguntas que pueden resultar interesantes: 
1. ¿Cómo son las primeras filas de este data set? 
2. ¿Cómo son los nombres de columnas? 
3. ¿Qué tipos de datos contiene cada columna? ¿Están todos bien? 
4. ¿Hay valores nulos? ¿Los completo? 
5. ¿Los datos son coherentes? Por ejemplo, si estoy trabajando con una variable que la edad de un individuo, los valores son siempre positivos? Tienen un límite máximo? 
6. ¿Hay coherencia entre grupos de datos? Por ejemplo, si estoy trabajando con un datasets que contiene niños en edad escolar diferenciados según si están estudiando primerio o secundario, la edad promedio de primario es menor a la edad promedio de secundario? 

## Ejemplo: explorando un dataset

In [ ]:
import pandas as pd
import numpy as np
url='https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv'
df= pd.read_csv(url, delimiter=',', header=None)
print(df.head(10))

In [ ]:
print(df.shape)

print('Inspeccion de Cantidad de filas y columnas:')
print('Cantidad de Filas:',df.shape[0])
print('Cantidad de Columnas:',df.shape[1])

## Qué significan las variables? 

### Variables incorporadas

0. Numero de veces embarazada (NEMB).
1. Concentracion de plasma de glucosa (GLU) 
2. Presion arterial diastolica en mm Hg (PART).
3. Grosor de piel en triceps en mm (GROS).
4. 2-Hour serum insulin en mu U/ml (HUR).
5. BMI (peso kg/(altura en m)^2 en (BMI).
6. Funcion de prediccion de Diabetes (FPRED)
7. Edad (años)  (AGE).
8. Variable de clase (0 or 1)  (CLASS).

In [ ]:
# Renombramos las columnas
nombres_columnas = {0: "NEMB", 
                    1: "GLU", 
                    2: "PART",
                    3:"GROS",
                    4:"HUR",
                    5:"BMI",
                    6:"FPRED",
                    7:"AGE",
                    8:"CLASS"}

In [ ]:
df = df.rename(columns = nombres_columnas)
print(df.head())

## Un poco de manipulación de datos

In [ ]:
# Cómo hago si solo quiero analizar las personas mayores a 25 años? 
filtro_aplicar = df['AGE'] > 25
print(filtro_aplicar)

In [ ]:
df_mas25 = df.loc[filtro_aplicar,:]
print(df_mas25)

In [ ]:
# Directo en la misma sentencia: 
df_mas25 = df.loc[df['AGE'] > 25,:]
print(df_mas25)

In [ ]:
# Y si quiero elegir algunas columnas? 
print(df.loc[:,['AGE','HUR']])

In [ ]:
# Y si quiero aplicar filtro y elegir columnas a la vez? 
filtro = df['AGE'] > 25
print(df.loc[filtro,['AGE','HUR']])

In [ ]:
# Y si quiero aplicar varios filtros a la vez? 
filtro_1 = df['AGE'] > 25
filtro_2 = df['HUR'] < 150
print(df.loc[(filtro_1) & (filtro_2),['AGE','HUR']])


In [ ]:
# Y si quiero aplicar varios que ocurra una cosa o la otra? 
filtro_1 = df['AGE'] > 25
filtro_2 = df['HUR'] < 150
print(df.loc[(filtro_1) | (filtro_2),['AGE','HUR']])

## Arrancando a explorar nulos

In [ ]:
# Usando el método info() para resumir el dataset
print(df.info())

In [ ]:
# Aparentemente no hay nulos, verifiquemos: .sum sobre una variable booleana me suma los trues.
print('Detalle de Nulos')
print(df.isnull().sum())

In [ ]:
# Usando el método describe() para explorar el dataset:
print(df.describe().round(2))

In [ ]:
# Vemos muchos ceros en variables que no tiene sentido
nun_missing = (df[['GLU','PART','GROS','HUR','BMI']] == 0).sum()
print(nun_missing)
print('\n')
print(df.isnull().sum())

In [ ]:
# Para darle tratamiento como nulo, reemplazamos los '0' con 'nan'
df[['GLU','PART','GROS','HUR','BMI']] = df[['GLU','PART','GROS','HUR','BMI']].replace(0, np.nan)
# Cuántos nulos me quedaron en cada variable?
print('Cantidad de nulos despues de reemplazar los 0 por nan')
print('\n')
print(df.isnull().sum())

In [ ]:
# Ver los nan en el df
df.head()

## Tratamiento de nulos
Al momento de trabajar con datos nulos, hay varias opciones: 
1. Eliminar del dataset todas las filas que contienen algún nulo (no suele ser lo mejor, pero en algún caso puede ser conveniente)
2. Imputar los nulos con algún valor (puede ser valor aleatorio, métrica, o resultado de algún modelo para nulos):
    * Forma manual: calculando la métrica que me interese para el reemplazo, e indicando que cada vez que aparezca un nulo inserte ese valor (por ejemplo, reemplazar con la media de la variable)
    * SimpleImputer: es una clase de preprocesamiento de la librería Scikit-learn que se usa para el manejo de datos faltantes. 

### Caso 1: Eliminar registros con datos nulos 

In [ ]:
# Usar el método dropna: 
# El parámetro inplace se utiliza para especificar si deseas que la operación de eliminación 
# de valores faltantes se realice directamente en el DataFrame original 
# o si deseas que se cree un nuevo DataFrame con los valores faltantes eliminados.
df_eliminado = df.dropna(inplace = False)
# ¿Cómo quedó el dataframe?
print(df_eliminado.shape)
print(df.shape)

### Caso 2: Reemplazar de forma manual - RECOMENDADO-

In [ ]:
# Vamos a elegir el promedio de cada columna para reemplazar 
promedios = df.mean().round()
print(promedios)

In [ ]:
# Uso el método fillna para reemplazar: 
df_reempl = df.fillna(promedios,
                      inplace = False)

In [ ]:
print('df Original:')
print(df.head())
print('\n')
print('df reemplazado:')
print(df_reempl.head())



### Caso 3: Reemplazar usando SimpleImputer

In [ ]:

pip install TensorFlow

In [ ]:
# Antes que nada, abro las librerías:
from sklearn.impute import SimpleImputer
import numpy as np 

In [ ]:
# Pasos a seguir:
# 1) Crear un numpy array con los valores
valores = df.values
# 2) Definir el imputador
imputador = SimpleImputer(missing_values = np.nan, 
                          strategy='mean')

# Algunas estrategias: 
# median, 
# most_frequent, 
# constant (indicando con el parámetro 'fill_value' el valor a asignar)

# 3) Transformar el dataset
transformados = imputador.fit_transform(valores)
print(type(transformados))

In [ ]:
# Transformo el ndarray en dataframe 
transformados = pd.DataFrame(transformados)
print(transformados.head().round(2))

In [ ]:
# Renombro las columnas 
transformados = transformados.rename(columns = nombres_columnas)
print(transformados)

## Agrupaciones

In [ ]:
print(df.sum())

In [ ]:
print(df[['NEMB','GLU']].mean())

In [ ]:
print(df[['NEMB','AGE']].median())

In [ ]:
# Usemos la variable "CLASS" para presentar algunas agrupaciones
print(df.groupby('CLASS').size())

In [ ]:
# Medidas de agregación que dependen de otras variables
print(df.groupby('CLASS')[['BMI','AGE','NEMB']].mean())

In [ ]:
# Varias medidas de agregación 
agrupaciones = df.groupby('CLASS').agg(
    
    min_AGE = ('AGE', 'min'),
    mean_AGE = ('AGE', 'mean'),
    max_AGE = ('AGE', 'max')
     
     )

print(agrupaciones.round())

In [ ]:
# Generar una nueva variable para agrupar  -CREAR UNA NUEVA COLUMNA EN BASE A OTRA COLUMNA
bins = [0,25,45,float('inf')]
labels = ['Joven','Mediana','Mayor']
# Python va a crear una columna usando el df.['NOMBRE']=
df['Edad_Agrupada'] = pd.cut(df['AGE'], 
                             bins = bins, 
                             labels = labels, 
                             right = False)
print(df[['Edad_Agrupada','AGE']].head())

In [ ]:
df.head()

In [ ]:
df.['Edad_Agrupada'].value_counts()

In [148]:
# Calcular NEMB y BMI promedio por grupo de edad 

print(df.groupby('Edad_Agrupada')[['NEMB','BMI']].mean())


                   NEMB        BMI
Edad_Agrupada                     
Joven          1.493151  30.856808
Mediana        4.240385  33.438983
Mayor          6.481203  31.965649


C:\Users\QY539GE\AppData\Local\Temp\ipykernel_1952\2400299143.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('Edad_Agrupada')[['NEMB','BMI']].mean())


In [149]:
# Calcular porcentaje de "CLASS" por grupo de edad 
cantidad = df.groupby('Edad_Agrupada')['CLASS'].count()
cantidad_1 = df.groupby('Edad_Agrupada')['CLASS'].sum()
cantidad_1 / cantidad

C:\Users\QY539GE\AppData\Local\Temp\ipykernel_1952\3571262087.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cantidad = df.groupby('Edad_Agrupada')['CLASS'].count()
C:\Users\QY539GE\AppData\Local\Temp\ipykernel_1952\3571262087.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cantidad_1 = df.groupby('Edad_Agrupada')['CLASS'].sum()


Edad_Agrupada
Joven      0.141553
Mediana    0.411058
Mayor      0.496241
Name: CLASS, dtype: float64

In [150]:
# Pero más simple: 
print(df.groupby('Edad_Agrupada')['CLASS'].mean())

Edad_Agrupada
Joven      0.141553
Mediana    0.411058
Mayor      0.496241
Name: CLASS, dtype: float64


C:\Users\QY539GE\AppData\Local\Temp\ipykernel_1952\3932878302.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('Edad_Agrupada')['CLASS'].mean())
